In [2]:
import qiskit as qk
import numpy as np
import scipy.linalg as lin
import math
from qiskit.quantum_info import Statevector , Operator , partial_trace , DensityMatrix
from qiskit.circuit.library.standard_gates import HGate , XGate
from qiskit.circuit.library import GlobalPhaseGate
from qiskit.extensions import Initialize , UnitaryGate
from qiskit.providers.aer import AerSimulator

/var/folders/y7/l21gv11s0fl71s2rjkzl_w1r0000gn/T/ipykernel_48554/21233476.py:8: DeprecationWarning: The qiskit.extensions module is deprecated since Qiskit 0.46.0. It will be removed in the Qiskit 1.0 release.
  from qiskit.extensions import Initialize , UnitaryGate
/var/folders/y7/l21gv11s0fl71s2rjkzl_w1r0000gn/T/ipykernel_48554/21233476.py:9: DeprecationWarning: Importing from 'qiskit.providers.aer' is deprecated. Import from 'qiskit_aer' instead, which should work identically.
  from qiskit.providers.aer import AerSimulator


In [146]:
# Build the quantum circuit
Q = 2
n = 2
i_qbits = qk.QuantumRegister(Q , 'i')
q_qbits = qk.QuantumRegister(Q , 'q')
k_qbits = qk.QuantumRegister(Q , 'k')
qc = qk.QuantumCircuit(k_qbits , i_qbits , q_qbits)

qc.h(q_qbits[0])
qc.ch(q_qbits[0] , q_qbits[1])

qc.ch(q_qbits[0] , k_qbits[0])
qc.ch(q_qbits[1] , k_qbits[1])

Omega = np.pi
Delta_t = 0.5
GDt = np.log(2)

In [9]:
print(qc.draw())

     ┌───┐               
q_0: ┤ H ├──■────■───────
     └───┘┌─┴─┐  │       
q_1: ─────┤ H ├──┼────■──
          └───┘┌─┴─┐  │  
k_0: ──────────┤ H ├──┼──
               └───┘┌─┴─┐
k_1: ───────────────┤ H ├
                    └───┘


In [147]:
Omega = np.pi/3.0
Delta_t = 0.5
GDt = np.log(2)

def Uc_Phi_Omega(Circuit , current_time , k_qbits_idx , q_qbits_idx , dagger= False):
    # Apply i\omega \sum_l=j^q (-1)^k_l
    # Omega is a global variable
    k_qbits = Circuit.qregs[k_qbits_idx]
    q_qbits = Circuit.qregs[q_qbits_idx]
    
    delta = -2.0*Omega*(Delta_t) # Dividing the angle by 2 for rz
    phase = 2.0*Omega*current_time

    if dagger:
        for l in range(len(k_qbits)-1 , - 1 , -1):
            if l > 0:
                for lk in range(l , 0 , -1):
                    Circuit.crz( delta/((lk+2)*(lk+1)) , q_qbits[l] , k_qbits[l-lk])
            Circuit.crz( -(l+1)*delta/(l+2) , q_qbits[l]  , k_qbits[l])
            #Circuit.rz(-1.0*phase , k_qbits[l])
    else:
        for l in range(len(k_qbits)):
            #Circuit.rz(phase , k_qbits[l])
            Circuit.crz((l+1)*delta/(l+2) , q_qbits[l] , k_qbits[l])
            if l > 0:
                for lk in np.arange(1 , l+1):
                    Circuit.crz(-delta/((lk+2)*(lk+1)) , q_qbits[l] , k_qbits[l - lk])

def Q_angle(Qq , i):
    angle = 0.0
    for q in np.arange(1 , Qq - i+1):
        angle += GDt**(q) / math.factorial(q + i)
    angle *= math.factorial(i)
    angle = np.sqrt(angle)

    return np.arctan(angle)

def UQ_circuit(Qq , n , dagger=False):
    L = int(np.log2(n))
    uq = qk.QuantumCircuit(Qq*L)
    if dagger:
        for q in range(Qq-1 , 0 , -1):
            uq.cry(-2.0*Q_angle(Qq , q) , (q-1)*L , q*L)
        uq.ry(-2.0*Q_angle(Qq, 0) , 0)  
    else:
        uq.ry(2.0*Q_angle(Qq, 0) , 0)
        for q in np.arange(1,Qq):
            uq.cry(2.0*Q_angle(Qq , q) , (q-1)*L , q*L) 
    return uq

def create_unitary(v):
    dim = v.size
    # Return identity if v is a multiple of e1
    if v[0][0] and not np.any(v[0][1:]):
        return np.identity( dim )
    e1 = np.zeros( dim )
    e1[0] = 1
    w = v/np.linalg.norm(v) - e1
    return np.identity(dim) - 2*((np.dot(w.T, w))/(np.dot(w, w.T)))

def UnitMap(vec , dagger=False):
    N = vec[0].size + 1
    U = np.zeros((N,N))
    U[0][0] = 1.0
    unit = create_unitary(vec)
    U[1: , 1:] = unit
    if dagger:
        U = U.conj().T
    return UnitaryGate(U)

def B_prepare(Circuit , kq_qbits_idx , iq_qbits_idx , q_qbits_idx ,  dagger=False):
        n = 2
        mapping_vec = np.zeros((1 , n-1))
        for i in np.arange(1,n):
            mapping_vec[0][i-1] = np.sqrt( 3.0 )
        mapping_vec[0] = mapping_vec[0]/np.sqrt( np.dot(mapping_vec[0],mapping_vec[0]) )
            
        L = int( np.log2(n) )
        iq_qbits = Circuit.qregs[iq_qbits_idx]
        kq_qbits = Circuit.qregs[kq_qbits_idx]
        q_qbits = Circuit.qregs[q_qbits_idx]

        Umapdag = UnitMap(mapping_vec , True)
        Umap = UnitMap(mapping_vec , False)
        
        if dagger:
            for i in range(Q-1 , -1 , -1):
                Circuit.append(Umapdag , iq_qbits[list(np.arange(i*L , (i+1)*L))])
            for i in range(Q-1 , -1 , -1):
                Circuit.ch(iq_qbits[i*L] , kq_qbits[i])
            for i in range(Q-1 , -1 , -1):
                Circuit.cx(iq_qbits[i*L] , q_qbits[i])
            Circuit.append(UQ_circuit(Q , n , dagger) , iq_qbits)

        else:
            Circuit.append(UQ_circuit(Q , n , dagger) , iq_qbits)
            for i in range(Q):
                Circuit.ch(iq_qbits[i*L] , kq_qbits[i])
            for i in range(Q):
                Circuit.cx(iq_qbits[i*L] , q_qbits[i])
            for i in range(Q):
                Circuit.append(Umap , iq_qbits[list(np.arange(i*L , (i+1)*L))])

def Uc_P( Circuit , ctr_qbits_idx , targ_qbits_idx , i_sub_idx , dagger):
# Assuming i_sub_idx refers to a specific block of register of size log2(n) qubits
    if i_sub_idx > 0:
        ctr_qbits = Circuit.qregs[ctr_qbits_idx]
        targ_qbits = Circuit.qregs[targ_qbits_idx]
        L = int( np.log2(n) )
        N_qbits = n + Q * L # Total qubits = z qubits + i_q qubits + k_q qubits
        
        ctr_qbits_subset = ctr_qbits[(i_sub_idx-1)*L:i_sub_idx*L]
        # Making the controlled XX gate for the specific number of controlled qubits
        cxxcirc = qk.QuantumCircuit(2 , name='cXX')
        cxxcirc.x( range(2) )
        cxxGate = cxxcirc.to_gate()
        cxxGate = cxxGate.control(L)
        
        if dagger:
            for i in np.arange(n-1 , 0 , -1):
                ibin = bin(i)[2:]
                zer_ctrs = [x for x in range(L) if(int(ibin[x]) == 0)]
                for j in range(len(zer_ctrs)):
                    Circuit.x( ctr_qbits_subset[zer_ctrs[j]] )
                Circuit.append( cxxGate , ctr_qbits_subset[:] + targ_qbits[i-1:i+1] )
                for j in np.arange(len(zer_ctrs)-1 , -1 , -1):
                    Circuit.x( ctr_qbits_subset[zer_ctrs[j]] )
        else:
            for i in np.arange(1,n):
                ibin = bin(i)[2:]
                #ibin = (Lctrs-len(ibin))*'0' +ibin 
                zer_ctrs = [x for x in range(L) if(int(ibin[x]) == 0)]
                for j in range(len(zer_ctrs)):
                    Circuit.x( ctr_qbits_subset[zer_ctrs[j]] )
                # This part needs to be fixed.. We need to 
                Circuit.append( cxxGate , ctr_qbits_subset[:] + targ_qbits[i-1:i+1] )
                for j in range(len(zer_ctrs)):
                    Circuit.x( ctr_qbits_subset[zer_ctrs[j]] )
                    
# UC_Phi(iq_qbits , z_qbits , q) generates the E_z_iq and E_z_ij related phases on |z>
def Uc_Phi(Circuit  , iq_qbits_idx , z_qbits_idx , q_qbits_idx , dagger=False):
    iq_qbits = Circuit.qregs[iq_qbits_idx]
    q_qbits = Circuit.qregs[q_qbits_idx]
    z_qbits = Circuit.qregs[z_qbits_idx]
    liq = iq_qbits.size
    if dagger:
        # Circuit.append( Diagonal_U0( Longit_h , Vx , Delta_t ) , z_qbits )
        U0_q_ctrl( Circuit , -Delta_t*Q , q_qbits , z_qbits , False)
        for j in range(Q , 0 , -1):
            Uc_P( Circuit , iq_qbits_idx , z_qbits_idx , j , dagger )
            U0_q_ctrl( Circuit , Delta_t , q_qbits , z_qbits , False )

            Circuit.cp( np.pi , q_qbits[j-1] , z_qbits[0] )
            Circuit.crz( -np.pi , q_qbits[j-1] , z_qbits[0] )
    else:
        for j in np.arange(1,Q+1):
            Circuit.crz( np.pi , q_qbits[j-1] , z_qbits[0] )
            Circuit.cp( -np.pi , q_qbits[j-1] , z_qbits[0] )

            #U0_q_ctrl( Circuit , Delta_t , q_qbits , z_qbits , True )
            Uc_P( Circuit , iq_qbits_idx , z_qbits_idx , j , dagger )
        #U0_q_ctrl( Circuit , -Delta_t*Q , q_qbits , z_qbits , True )


In [95]:
qc = qk.QuantumCircuit(q_qbits , k_qbits)
Uc_Phi_Omega(qc , 0 , 1 , 0 , False)
print(qc.draw())

                                                  
q_0: ──────────────■──────────────────────────────
                   │                              
q_1: ──────────────┼────────────■───────────■─────
     ┌───────┐┌────┴─────┐      │      ┌────┴────┐
k_0: ┤ Rz(0) ├┤ Rz(-π/2) ├──────┼──────┤ Rz(π/6) ├
     ├───────┤└──────────┘┌─────┴─────┐└─────────┘
k_1: ┤ Rz(0) ├────────────┤ Rz(-2π/3) ├───────────
     └───────┘            └───────────┘           


In [59]:
#state = Statevector.from_label('0'*2*Q)
state = np.array([1 , 0 , 0 , 0 , 1 , 1 , 0 , 0 , 0 , 0 , 0 , 0, 1 , 1 , 1 , 1])
state = np.array([1 , 1, 0, 0, 0 , 1, 1 , 0, 0 , 0, 1, 0 , 0 , 0 , 1 , 1])
state = np.array([1 , 1, 0, 1, 0 , 1, 0 , 1, 0 , 0, 0, 1 , 0 , 0 , 0 , 1])

#state = np.array([1 , 1 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0, 0 , 0 , 0 , 0])
state = state/np.linalg.norm(state)
state = Statevector(state)
final = state.evolve(qc)

In [57]:
print(f'The final state is {final}')
print(f'The norm of the final state is {np.linalg.norm(final)}')

The final state is Statevector([3.77964473e-01+0.j        , 2.67261242e-01+0.26726124j,
             0.00000000e+00+0.j        , 0.00000000e+00+0.j        ,
             0.00000000e+00+0.j        , 2.67261242e-01-0.26726124j,
             9.78244040e-02+0.36508565j, 0.00000000e+00+0.j        ,
             0.00000000e+00+0.j        , 0.00000000e+00+0.j        ,
             9.78244040e-02-0.36508565j, 0.00000000e+00+0.j        ,
             0.00000000e+00+0.j        , 0.00000000e+00+0.j        ,
             2.67261242e-01-0.26726124j, 6.72377924e-17-0.37796447j],
            dims=(2, 2, 2, 2))
The norm of the final state is 1.0


$|00\rangle |00\rangle +  (e^{i\omega t/2}|00\rangle+e^{-i\omega t/2}|01\rangle)|01\rangle +  (e^{i\omega t}|00\rangle + e^{i\omega t/3}|01 \rangle + e^{-i\omega t/3}|10\rangle +e^{-i\omega t}|11\rangle)|11\rangle$

In [140]:
state_theory = np.zeros((1,2**(3*Q)), dtype='complex')[0]
state_theory[0] = 1.0
state_theory[20] = np.exp(1.0j*Omega*Delta_t/2.0)
state_theory[21] = np.exp(-1.0j*Omega*Delta_t/2.0)
state_theory[63] = np.exp(-1.0j*Omega*Delta_t)
state_theory[62] = np.exp(-1.0j*Omega*Delta_t/3.0)
state_theory[61] = np.exp(1.0j*Omega*Delta_t/3.0)
state_theory[60] = np.exp(1.0j*Omega*Delta_t)
state_theory = state_theory/np.linalg.norm(state_theory)
state_theory = Statevector(state_theory)

print(f'the theoretical state is {state_theory}')

the theoretical state is Statevector([0.37796447+0.j        , 0.        +0.j        ,
             0.        +0.j        , 0.        +0.j        ,
             0.        +0.j        , 0.        +0.j        ,
             0.        +0.j        , 0.        +0.j        ,
             0.        +0.j        , 0.        +0.j        ,
             0.        +0.j        , 0.        +0.j        ,
             0.        +0.j        , 0.        +0.j        ,
             0.        +0.j        , 0.        +0.j        ,
             0.        +0.j        , 0.        +0.j        ,
             0.        +0.j        , 0.        +0.j        ,
             0.36508565+0.0978244j , 0.36508565-0.0978244j ,
             0.        +0.j        , 0.        +0.j        ,
             0.        +0.j        , 0.        +0.j        ,
             0.        +0.j        , 0.        +0.j        ,
             0.        +0.j        , 0.        +0.j        ,
             0.        +0.j        , 0.        +0.j        ,

In [60]:
#final == state_theory
print(f'Tthe real state is {final}')

Tthe real state is Statevector([3.77964473e-01+0.j        , 2.67261242e-01+0.26726124j,
             0.00000000e+00+0.j        , 6.72377924e-17+0.37796447j,
             0.00000000e+00+0.j        , 2.67261242e-01-0.26726124j,
             0.00000000e+00+0.j        , 3.27326835e-01+0.18898224j,
             0.00000000e+00+0.j        , 0.00000000e+00+0.j        ,
             0.00000000e+00+0.j        , 3.27326835e-01-0.18898224j,
             0.00000000e+00+0.j        , 0.00000000e+00+0.j        ,
             0.00000000e+00+0.j        , 6.72377924e-17-0.37796447j],
            dims=(2, 2, 2, 2))


In [26]:
X2 = np.kron(np.eye(2) , np.array([[0 , 1] , [1 , 0]]))

print(np.dot(X2 , np.array([1, 0, 0 , 0])))

[0. 1. 0. 0.]


In [141]:
i_qbits = qk.QuantumRegister(Q , 'i')
q_qbits = qk.QuantumRegister(Q , 'q')
k_qbits = qk.QuantumRegister(Q , 'k')
Circ = qk.QuantumCircuit(k_qbits , i_qbits , q_qbits)
#B_prepare( Circ , 0 , 1 , 2 ,  False )
Uc_Phi_Omega(Circ , 0.0 , 0 , 2 , False)

state = np.zeros((1,2**(3*Q)))[0]
state[0] = 1.0
state[20] = 1.0
state[21] = 1.0
state[63] = 1.0
state[62] = 1.0
state[61] = 1.0
state[60] = 1.0
state = state/np.linalg.norm(state)
state = Statevector(state)
#state = Statevector.from_label('0'*3*Q)
final = state.evolve(Circ)
#for i in range(len(final)):
#    if abs(final[i])>1E-6:
#        print('The index is ', i)
print(final/np.linalg.norm(final))
#print(f'The norm of the final state is {np.linalg.norm(final)}')
print(f'the theoretical state is {state_theory}')



Statevector([0.37796447+0.j        , 0.        +0.j        ,
             0.        +0.j        , 0.        +0.j        ,
             0.        +0.j        , 0.        +0.j        ,
             0.        +0.j        , 0.        +0.j        ,
             0.        +0.j        , 0.        +0.j        ,
             0.        +0.j        , 0.        +0.j        ,
             0.        +0.j        , 0.        +0.j        ,
             0.        +0.j        , 0.        +0.j        ,
             0.        +0.j        , 0.        +0.j        ,
             0.        +0.j        , 0.        +0.j        ,
             0.36508565+0.0978244j , 0.36508565-0.0978244j ,
             0.        +0.j        , 0.        +0.j        ,
             0.        +0.j        , 0.        +0.j        ,
             0.        +0.j        , 0.        +0.j        ,
             0.        +0.j        , 0.        +0.j        ,
             0.        +0.j        , 0.        +0.j        ,
             0.        +

In [133]:
print(Circ.draw())

     ┌──────────┐             ┌─────────┐
k_0: ┤ Rz(-π/2) ├─────────────┤ Rz(π/6) ├
     └────┬─────┘┌───────────┐└────┬────┘
k_1: ─────┼──────┤ Rz(-2π/3) ├─────┼─────
          │      └─────┬─────┘     │     
i_0: ─────┼────────────┼───────────┼─────
          │            │           │     
i_1: ─────┼────────────┼───────────┼─────
          │            │           │     
q_0: ─────■────────────┼───────────┼─────
                       │           │     
q_1: ──────────────────■───────────■─────
                                         


0.3779644730092272

In [150]:
i_qbits = qk.QuantumRegister(2 , 'i')
z_qbits = qk.QuantumRegister(2 , 'z')
q_qbits = qk.QuantumRegister(2 , 'q')
Circ = qk.QuantumCircuit(i_qbits , z_qbits , q_qbits)
Uc_Phi(Circ  , 0 , 1 , 2 , False)


state = np.array(np.zeros((1 , 2**(6)))[0])
state[51] = 1.0
state = Statevector(state)

final = state.evolve(Circ)

for i in range(len(final)):
    if abs(final[i]) > 1E-6:
        print(f'the value is {final[i]} and the index is {i}')

print(final)

the value is (-1-1.2246467991473532e-16j) and the index is 51
Statevector([ 0.+0.0000000e+00j,  0.+0.0000000e+00j,  0.+0.0000000e+00j,
              0.+0.0000000e+00j,  0.+0.0000000e+00j,  0.+0.0000000e+00j,
              0.+0.0000000e+00j,  0.+0.0000000e+00j,  0.+0.0000000e+00j,
              0.+0.0000000e+00j,  0.+0.0000000e+00j,  0.+0.0000000e+00j,
              0.+0.0000000e+00j,  0.+0.0000000e+00j,  0.+0.0000000e+00j,
              0.+0.0000000e+00j,  0.+0.0000000e+00j,  0.+0.0000000e+00j,
              0.+0.0000000e+00j,  0.+0.0000000e+00j,  0.+0.0000000e+00j,
              0.+0.0000000e+00j,  0.+0.0000000e+00j,  0.+0.0000000e+00j,
              0.+0.0000000e+00j,  0.+0.0000000e+00j,  0.+0.0000000e+00j,
              0.+0.0000000e+00j,  0.+0.0000000e+00j,  0.+0.0000000e+00j,
              0.+0.0000000e+00j,  0.+0.0000000e+00j,  0.+0.0000000e+00j,
              0.+0.0000000e+00j,  0.+0.0000000e+00j,  0.+0.0000000e+00j,
              0.+0.0000000e+00j,  0.+0.0000000e+00j,  0.+0.000